### Importando as Bibliotecas e Funções

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [ ]:
def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df

def plot_profit_acu(dataframe, title_text):
    dataframe['Profit_acu'] = dataframe.Profit.cumsum()
    dataframe['Investimento'] = 1
    n_apostas = dataframe.shape[0]
    profit = round(dataframe.Profit_acu.tail(1).item(), 2)
    dataframe['Investimento_acu'] = dataframe.Investimento.cumsum()
    ROI = round(((dataframe.Profit_acu.tail(1) / dataframe.Investimento_acu.tail(1)) * 100).item(), 2)
    drawdown = dataframe['Profit_acu'] - dataframe['Profit_acu'].cummax()
    drawdown_maximo = round(drawdown.min(), 2)
    winrate_medio = round((dataframe['Profit'] > 0).mean() * 100, 2)
    desvio_padrao = round(dataframe['Profit'].std(), 2)

    ax = dataframe.Profit_acu.plot(title=title_text, xlabel='Entradas', ylabel='Stakes')
    ax.set_title(title_text)
    ax.set_xlabel('Entradas')
    ax.set_ylabel('Stakes')

    print("Metodo:", title_text)
    print("Profit:", profit, "stakes em", n_apostas, "jogos")
    print("ROI:", ROI, "%")
    print("Drawdown Maximo Acumulado:", drawdown_maximo)
    print("Winrate Medio:", winrate_medio, "%")
    print("Desvio Padrao:", desvio_padrao)
    print("")

    plt.show()

### Importando a Base de Dados

In [ ]:
df = pd.read_csv("https://www.football-data.co.uk/new/BRA.csv")
df = df[['Country','League','Season','Date','Time','Home','Away',
         'HG','AG','Res','PSCH','PSCD','PSCA']]
df.columns = ['Country','League','Season','Date','Time','Home','Away',
              'Goals_H','Goals_A','Result','Odd_H','Odd_D','Odd_A']
df = drop_reset_index(df)
display(df)

### Criando as Variáveis

##### Coeficiente de Variação das Odds do Match Odds

In [ ]:
desvio_padrao = df[['Odd_H','Odd_D','Odd_A']].std(axis=1)
media = df[['Odd_H','Odd_D','Odd_A']].mean(axis=1)
CV_Odds = desvio_padrao / media
df['CV_Odds'] = CV_Odds

##### Média de Pontos (PPG)

In [ ]:
n_per = 5

df['Ptos_H'] = np.where(df['Goals_H'] >  df['Goals_A'], 3,
               np.where(df['Goals_H'] == df['Goals_A'], 1, 0))


df['Ptos_A'] = np.where(df['Goals_H'] > df['Goals_A'], 0,
               np.where(df['Goals_H'] == df['Goals_A'], 1, 3))

df['Media_Ptos_H'] = df.groupby('Home')['Ptos_H'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_Ptos_A'] = df.groupby('Away')['Ptos_A'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_Ptos_H'] = df.groupby('Home')['Media_Ptos_H'].shift(1)
df['Media_Ptos_A'] = df.groupby('Away')['Media_Ptos_A'].shift(1)

df['DesvPad_Ptos_H'] = df.groupby('Home')['Ptos_H'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_Ptos_A'] = df.groupby('Away')['Ptos_A'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_Ptos_H'] = df.groupby('Home')['DesvPad_Ptos_H'].shift(1)
df['DesvPad_Ptos_A'] = df.groupby('Away')['DesvPad_Ptos_A'].shift(1)

df['CV_Ptos_H'] = df['DesvPad_Ptos_H'] / df['Media_Ptos_H']
df['CV_Ptos_A'] = df['DesvPad_Ptos_A'] / df['Media_Ptos_A']

##### Média Gols Marcados

In [ ]:
df['Media_GM_H'] = df.groupby('Home')['Goals_H'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GM_A'] = df.groupby('Away')['Goals_A'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GM_H'] = df.groupby('Home')['Media_GM_H'].shift(1)
df['Media_GM_A'] = df.groupby('Away')['Media_GM_A'].shift(1)

df['DesvPad_GM_H'] = df.groupby('Home')['Goals_H'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GM_A'] = df.groupby('Away')['Goals_A'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GM_H'] = df.groupby('Home')['DesvPad_GM_H'].shift(1)
df['DesvPad_GM_A'] = df.groupby('Away')['DesvPad_GM_A'].shift(1)

df['CV_GM_H'] = df['DesvPad_GM_H'] / df['Media_GM_H']
df['CV_GM_A'] = df['DesvPad_GM_A'] / df['Media_GM_A']

##### Média de Gols Sofridos

In [ ]:
df['Media_GS_H'] = df.groupby('Home')['Goals_A'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GS_A'] = df.groupby('Away')['Goals_H'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GS_H'] = df.groupby('Home')['Media_GS_H'].shift(1)
df['Media_GS_A'] = df.groupby('Away')['Media_GS_A'].shift(1)

df['DesvPad_GS_H'] = df.groupby('Home')['Goals_A'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GS_A'] = df.groupby('Away')['Goals_H'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GS_H'] = df.groupby('Home')['DesvPad_GS_H'].shift(1)
df['DesvPad_GS_A'] = df.groupby('Away')['DesvPad_GS_A'].shift(1)

df['CV_GS_H'] = df['DesvPad_GS_H'] / df['Media_GS_H']
df['CV_GS_A'] = df['DesvPad_GS_A'] / df['Media_GS_A']

##### Média de Saldo de Gols

In [ ]:
df['SG_H'] = df['Goals_H'] - df['Goals_A']
df['SG_A'] = df['Goals_A'] - df['Goals_H']

df['Media_SG_H'] = df.groupby('Home')['SG_H'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_A'] = df.groupby('Away')['SG_A'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_H'] = df.groupby('Home')['Media_SG_H'].shift(1)
df['Media_SG_A'] = df.groupby('Away')['Media_SG_A'].shift(1)

df['DesvPad_SG_H'] = df.groupby('Home')['SG_H'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_A'] = df.groupby('Away')['SG_A'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_H'] = df.groupby('Home')['DesvPad_SG_H'].shift(1)
df['DesvPad_SG_A'] = df.groupby('Away')['DesvPad_SG_A'].shift(1)

df['CV_SG_H'] = df['DesvPad_SG_H'] / df['Media_SG_H']
df['CV_SG_A'] = df['DesvPad_SG_A'] / df['Media_SG_A']

##### Análise dos Resultados

In [ ]:
df['Back_Home'] = np.where((df['Goals_H'] >  df['Goals_A']), 1, 0)
df['Back_Draw'] = np.where((df['Goals_H'] == df['Goals_A']), 1, 0)
df['Back_Away'] = np.where((df['Goals_H']  < df['Goals_A']), 1, 0)

df['Profit_Back_H'] = np.where((df['Back_Home'] == 1), df['Odd_H']-1, -1)
df['Profit_Back_D'] = np.where((df['Back_Draw'] == 1), df['Odd_D']-1, -1)
df['Profit_Back_A'] = np.where((df['Back_Away'] == 1), df['Odd_A']-1, -1)

In [ ]:
flt = (df.Season == 2023) | (df.Season == 2024)
df = df[flt]
df = drop_reset_index(df)
display(df)

In [ ]:
df['Profit'] = df['Profit_Back_H']
plot_profit_acu(df, 'Back Home')

In [ ]:
df['Profit'] = df['Profit_Back_D']
plot_profit_acu(df, 'Back Draw')

In [ ]:
df['Profit'] = df['Profit_Back_A']
plot_profit_acu(df, 'Back Away')

### Análise Exploratória dos Dados

In [ ]:
# Informações básicas da base de dados
df.info()

In [ ]:
# Estatísticas descritivas das variáveis numéricas
df.describe()

In [ ]:
# Ajustar o estilo dos gráficos
plt.style.use('seaborn-darkgrid')

# Distribuição dos resultados (Vitória Casa, Empate, Vitória Visitante)
result_counts = df['Result'].value_counts()
plt.figure(figsize=(8,5))
sns.barplot(x=result_counts.index, y=result_counts.values, palette='Blues_d')
plt.title('Distribuição dos Resultados')
plt.xlabel('Resultado')
plt.ylabel('Contagem')
plt.show()

In [ ]:
# Função para calcular a média de gols marcados e sofridos
def calcular_media_gols(df, tipo):
    if tipo == "Home":
        media_gols_marcados = df.groupby('Home')['Goals_H'].mean()
        media_gols_sofridos = df.groupby('Home')['Goals_A'].mean()
    elif tipo == "Away":
        media_gols_marcados = df.groupby('Away')['Goals_A'].mean()
        media_gols_sofridos = df.groupby('Away')['Goals_H'].mean()

    media_gols = pd.DataFrame({
        f'Média_Gols_Marcados_{tipo}': media_gols_marcados,
        f'Média_Gols_Sofridos_{tipo}': media_gols_sofridos
    }).fillna(0)

    return media_gols

# Cálculo de médias de gols em casa e fora
media_gols_casa = calcular_media_gols(df, 'Home')
media_gols_fora = calcular_media_gols(df, 'Away')

# Unir os resultados das médias de gols para cada time
media_gols_gerais = pd.concat([media_gols_casa, media_gols_fora], axis=1)

display(media_gols_gerais)

In [ ]:
# Histograma da distribuição de gols marcados pelos times da casa e visitantes
plt.figure(figsize=(12, 6))

# Gols dos times da casa
plt.subplot(1, 2, 1)
sns.histplot(df['Goals_H'], bins=10, kde=True, color='blue')
plt.title('Distribuição de Gols Marcados pelos Times da Casa')
plt.xlabel('Gols')
plt.ylabel('Frequência')

# Gols dos times visitantes
plt.subplot(1, 2, 2)
sns.histplot(df['Goals_A'], bins=10, kde=True, color='red')
plt.title('Distribuição de Gols Marcados pelos Times Visitantes')
plt.xlabel('Gols')
plt.ylabel('Frequência')

plt.tight_layout()
plt.show()

# Visualizando a distribuição de odds
plt.figure(figsize=(12, 6))

# Odds para vitória da casa
plt.subplot(1, 2, 1)
sns.histplot(df['Odd_H'], bins=15, kde=True, color='green')
plt.title('Distribuição das Odds para Vitórias da Casa')
plt.xlabel('Odds')
plt.ylabel('Frequência')

# Odds para vitória do visitante
plt.subplot(1, 2, 2)
sns.histplot(df['Odd_A'], bins=15, kde=True, color='green')
plt.title('Distribuição das Odds para Vitórias do Visitante')
plt.xlabel('Odds')
plt.ylabel('Frequência')

plt.tight_layout()
plt.show()

In [ ]:
colunas = ['Odd_H','Odd_D','Odd_A','CV_Odds',
           'Media_Ptos_H','Media_Ptos_A','DesvPad_Ptos_H','DesvPad_Ptos_A','CV_Ptos_H','CV_Ptos_A',
           'Media_GM_H','Media_GM_A','DesvPad_GM_H','DesvPad_GM_A','CV_GM_H','CV_GM_A',
           'Media_GS_H','Media_GS_A','DesvPad_GS_H','DesvPad_GS_A','CV_GS_H','CV_GS_A',
           'Media_SG_H','Media_SG_A','DesvPad_SG_H','DesvPad_SG_A','CV_SG_H','CV_SG_A',
           'Profit_Back_H']

df0 = df[colunas]
display(df0)

In [ ]:
# Correlação entre variáveis numéricas
plt.figure(figsize=(24,16))
corr_matrix = df0.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Matriz de Correlação entre Variáveis Numéricas')
plt.show()